# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


! pip3 install pyspark

In [1]:
! pip3 install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 7.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425347 sha256=9fa4bf4b5f04ad632a44457d9e9c82cfdd21ad2c5172d99772ad0927f9038806
  Stored in directory: /home/jovyan/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
sc = pyspark.SparkContext(appName='jupyter')
se = SparkSession(sc)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-10-07 16:32:09,483 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
!hadoop fs -put clickstream.csv clickstream.csv

In [4]:

# Load the data
clickstream_df = se.read.option("delimiter", "\t").csv("clickstream.csv", header=True, inferSchema=True)


# Show the first few rows
clickstream_df.show(10)

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584160|
|    562|       507|        page|    rabota|1695584166|
|    562|       507|       event|    rabota|1695584174|
+-------+----------+------------+----------+----------+
only showing top 10 rows



In [5]:
clickstream_df.createOrReplaceTempView("clickstream")
query = """SELECT * FROM clickstream LIMIT 100"""
filtered_df = se.sql(query)
filtered_df.show(10)

+-------+----------+----------+----------+----------+
|user_id|session_id|event_type|event_page| timestamp|
+-------+----------+----------+----------+----------+
|   4645|       935|      page|   archive|1696270588|
|   2251|       378|      page|    rabota|1696270591|
|   2222|       704|      page|    rabota|1696270600|
|   2366|       339|      page|     vklad|1696270605|
|   2233|       165|     event|    rabota|1696270605|
|   3253|       528|     event|      news|1696270612|
|    109|       433|     event|     bonus|1696270613|
|   4367|       106|     event|  internet|1696270619|
|   2222|       704|     event|    rabota|1696270628|
|   4304|       640|     event|  internet|1696270636|
+-------+----------+----------+----------+----------+
only showing top 10 rows



# Spark SQL

In [47]:

query = """
WITH FirstErrorTimestamps AS (
    SELECT user_id, session_id, MIN(timestamp) as error_timestamp
    FROM clickstream
    WHERE event_type LIKE '%error%'
    GROUP BY user_id, session_id
),
FilteredPages AS (
    SELECT c.user_id, c.session_id, c.event_page, c.timestamp
    FROM clickstream c
    LEFT JOIN FirstErrorTimestamps e ON c.user_id = e.user_id AND c.session_id = e.session_id
    WHERE c.event_type = 'page' 
    AND (c.timestamp < e.error_timestamp OR e.error_timestamp IS NULL)
),
AggregatedRoutes AS (
    SELECT user_id, session_id, 
           COLLECT_LIST(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) AS route_list, timestamp
    FROM FilteredPages
),
ranked_routes AS (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY user_id, session_ID ORDER BY timestamp DESC) as row_num
    FROM AggregatedRoutes
)
, final_table AS (
    SELECT user_id, session_id, CONCAT_WS('-', route_list) AS route_str, timestamp FROM ranked_routes
    WHERE row_num = 1
)

SELECT route_str as route, COUNT(*) count_ FROM final_table
GROUP BY route_str
ORDER BY count_ DESC
LIMIT 30
"""

routes_df = se.sql(query)
routes_df.show(30)


+--------------------+------+
|               route|count_|
+--------------------+------+
|                main|  8090|
|        main-archive|  1096|
|         main-rabota|  1039|
|       main-internet|   880|
|          main-bonus|   865|
|           main-news|   760|
|        main-tariffs|   669|
|         main-online|   584|
|          main-vklad|   514|
| main-archive-rabota|   167|
| main-rabota-archive|   167|
|  main-bonus-archive|   139|
|   main-rabota-bonus|   137|
|    main-news-rabota|   134|
|   main-bonus-rabota|   133|
|main-archive-inte...|   131|
|    main-rabota-news|   129|
|main-internet-rabota|   128|
|   main-archive-news|   125|
|main-rabota-internet|   123|
|main-internet-arc...|   123|
|  main-archive-bonus|   117|
|main-tariffs-inte...|   114|
| main-internet-bonus|   114|
|   main-news-archive|   112|
|  main-news-internet|   108|
|main-archive-tariffs|   103|
|  main-internet-news|   102|
|main-tariffs-archive|   102|
|           main-main|    94|
+---------

In [43]:
import glob
print(glob.glob("/home/jovyan/work/*"))

['/home/jovyan/work/sga-task.ipynb', '/home/jovyan/work/clickstream.csv', '/home/jovyan/work/hdfs-task.ipynb']


In [42]:
import os

In [36]:
os.getcwd()

'/home/jovyan/work'

In [45]:
routes_df.limit(30).write.csv('/home/jovyan/work/sql_solution.csv', sep='\t', header=True, mode='overwrite')

In [46]:
!ls

clickstream.csv  hdfs-task.ipynb  sga-task.ipynb


In [48]:
import pandas

In [49]:
query_result = se.sql(query)
query_pandas_df = query_result.toPandas()

In [59]:
query_pandas_df.head(30)

,route,count_
0,main,8090
1,main-archive,1096
2,main-rabota,1039
3,main-internet,880
4,main-bonus,865
5,main-news,760
6,main-tariffs,669
7,main-online,584
8,main-vklad,514
9,main-rabota-archive,167


In [54]:
query_pandas_df.to_csv('/home/jovyan/work/sql_solution.csv', sep="\t", index=False)

# RDD

In [16]:
clickstream_rdd = clickstream_df.rdd

In [18]:
from operator import add

In [55]:



# Get first error timestamp for each (user_id, session_id)
error_timestamps = (clickstream_rdd
                    .filter(lambda r: 'error' in r.event_type)
                    .map(lambda r: ((r.user_id, r.session_id), r.timestamp))
                    .reduceByKey(min)
                   )

# Filter out error events and those after first error for each session
valid_pages = (clickstream_rdd
               .filter(lambda r: r.event_type == 'page')
               .map(lambda r: ((r.user_id, r.session_id), r))
               .leftOuterJoin(error_timestamps)
               .filter(lambda x: x[1][1] is None or x[1][0].timestamp < x[1][1])
               .map(lambda x: x[1][0])
              )

# Create a session-route string for each event, then reduce to get distinct routes
routes = (valid_pages
          .map(lambda r: ((r.user_id, r.session_id), [r.event_page]))
          .reduceByKey(add)
          .map(lambda x: ("-".join(x[1]), 1))
          .reduceByKey(add)
         )

# Order by count descending
ordered_routes = routes.sortBy(lambda x: x[1], ascending=False)

# Take top 20
top_routes = ordered_routes.take(30)

# Display
for route, count in top_routes:
    print(f"{route}: {count}")

main: 8090
main-archive: 1092
main-rabota: 1038
main-internet: 880
main-bonus: 865
main-news: 759
main-tariffs: 668
main-online: 584
main-vklad: 513
main-archive-rabota: 167
main-rabota-archive: 167
main-bonus-archive: 139
main-rabota-bonus: 137
main-news-rabota: 134
main-bonus-rabota: 133
main-archive-internet: 131
main-rabota-news: 129
main-internet-rabota: 128
main-archive-news: 125
main-internet-archive: 123
main-rabota-internet: 123
main-archive-bonus: 117
main-tariffs-internet: 114
main-internet-bonus: 113
main-news-archive: 112
main-news-internet: 108
main-archive-tariffs: 103
main-internet-news: 102
main-tariffs-archive: 101
main-main: 94


In [57]:
rdd_spark_df = ordered_routes.toDF(["route", "count"])
rdd_pandas_df = rdd_spark_df.toPandas()

In [58]:
rdd_pandas_df.head(30).to_csv('/home/jovyan/work/rdd_solution.csv', sep="\t", index=False

,route,count
0,main,8090
1,main-archive,1092
2,main-rabota,1038
3,main-internet,880
4,main-bonus,865
5,main-news,759
6,main-tariffs,668
7,main-online,584
8,main-vklad,513
9,main-archive-rabota,167


# DF

In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F

# Get first error timestamp for each (user_id, session_id)
error_timestamps = (
    clickstream_df.filter(F.col("event_type").like("%error%"))
    .groupBy("user_id", "session_id")
    .agg(F.min("timestamp").alias("first_error_ts"))
)

# Join the error timestamps back to the original DataFrame and filter the valid pages
valid_pages = (
    clickstream_df.join(
        error_timestamps,
        on=["user_id", "session_id"],
        how="left_outer"
    )
    .filter(
        (F.col("event_type") == "page") 
        & ((F.col("timestamp") < F.col("first_error_ts")) | F.col("first_error_ts").isNull())
    )
)

# Define a window spec to create the routes
window_spec = (
    Window.partitionBy("user_id", "session_id")
    .orderBy("timestamp")
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

# Create the routes
routes = (
    valid_pages.withColumn("route", F.collect_list("event_page").over(window_spec))
    .groupBy("user_id", "session_id")
    .agg(F.max("route").alias("route"))
    .withColumn("route_str", F.expr("concat_ws('-', route)"))
)

# Count the occurrences of each route
route_counts = (
    routes.groupBy("route_str")
    .count()
    .orderBy(F.desc("count"))
)

# Display the top 20
route_counts.show(20)


+--------------------+-----+
|           route_str|count|
+--------------------+-----+
|                main| 8090|
|        main-archive| 1096|
|         main-rabota| 1039|
|       main-internet|  880|
|          main-bonus|  865|
|           main-news|  760|
|        main-tariffs|  669|
|         main-online|  584|
|          main-vklad|  514|
| main-archive-rabota|  167|
| main-rabota-archive|  167|
|  main-bonus-archive|  139|
|   main-rabota-bonus|  137|
|    main-news-rabota|  134|
|   main-bonus-rabota|  133|
|main-archive-inte...|  131|
|    main-rabota-news|  129|
|main-internet-rabota|  128|
|   main-archive-news|  125|
|main-rabota-internet|  123|
+--------------------+-----+
only showing top 20 rows

